In [1]:
import hydra
import os
os.environ['MUJOCO_GL'] = 'egl'
import warnings
warnings.filterwarnings('ignore')
from hydra.core.hydra_config import HydraConfig
from omegaconf import OmegaConf
from huggingface_hub import HfFileSystem
from huggingface_hub import hf_hub_download

import gym
import h5py
import imageio
import numpy as np
import torch
from termcolor import colored

from common.buffer import Buffer
from common.parser import parse_cfg
from common.seed import set_seed
from envs import make_env
from tdmpc2 import TDMPC2
from collections import defaultdict
from envs.wrappers.pixels import PixelWrapper

from tensordict import TensorDict
from mujoco import mjtVisFlag
from IPython.display import Video
from envs.myosuite import MYOSUITE_TASKS

In [92]:
class RobomimicBuffer(Buffer):
    def get_episode(self, episode_id):
        """Retrieve a TensorDict for a specific episode."""
        return self._buffer.storage[self._buffer.storage['episode'] == episode_id]

    def save_hdf5(self, path): 
        with h5py.File(path, 'w') as f:
            for episode_id in range(self.num_eps):
                episode_td = self.get_episode(episode_id)
                # Convert TensorDict to numpy and save to HDF5
                f["data/demo_{ep}"].attrs["num_samples"] = len(episode_td)
                for key, value in episode_td.items():
                    if isinstance(value, dict):
                        for sub_key, sub_value in value.items():
                            f.create_dataset(f'data/demo_{episode_id}/{key}/{sub_key}', data=sub_value.cpu().numpy())
                    else:
                        f.create_dataset(f'data/demo_{episode_id}/{key}', data=value.cpu().numpy())
                    f.create_dataset(f'data/demo_{episode_id}/{key}', data=value.cpu().numpy())



In [59]:
buffer = evaluate(cfg)

Task: myo-key-turn
Model size: default
Checkpoint: /home/krishnans/.cache/huggingface/hub/models--nicklashansen--tdmpc2/snapshots/8fb2a82efb3bae96941da440128fe1332e4394fd/myosuite/myo-hand-key-turn-1.pt


Evaluating agent on myo-key-turn:
Buffer capacity: 1,000
Storage required: 0.44 GB
Using CUDA memory for storage.
  myo-key-turn          	R: 1217.1  	S: 1.00


In [96]:
f = h5py.File("test.hdf5", 'w')

In [98]:
f.create_group('data')

<HDF5 group "/data" (0 members)>

In [110]:
len(ep0)

91

In [102]:
ep0 = buffer._buffer.storage[buffer._buffer.storage['episode']==0]

In [106]:

for key in ep0['obs'].keys():
    f.create_dataset(f'demo_0/obs/{key}', data=ep0['obs'][key].cpu().numpy())

In [107]:
ep0.items()

dict_items([('obs', TensorDict(
    fields={
        fixed_camera: Tensor(shape=torch.Size([91, 384, 384, 3]), device=cuda:0, dtype=torch.uint8, is_shared=True),
        success: Tensor(shape=torch.Size([91]), device=cuda:0, dtype=torch.bool, is_shared=True),
        vec_obs: Tensor(shape=torch.Size([91, 93]), device=cuda:0, dtype=torch.float32, is_shared=True)},
    batch_size=torch.Size([91]),
    device=cuda,
    is_shared=True)), ('reward', tensor([ 5.3710,  5.5485,  5.7160,  5.8861,  6.0586,  6.2297,  6.3981,  6.5738,
         6.7718, 10.9458, 11.1110, 11.2819, 11.4624, 11.6285, 11.7529, 11.8715,
        11.9860, 12.1239, 12.2574, 12.3893, 12.5116, 12.6253, 12.7352, 12.8283,
        12.9650, 13.1114, 13.2290, 13.3091, 13.3812, 13.4566, 13.5617, 13.6425,
        13.7482, 13.8462, 13.8774, 14.0281, 14.2355, 14.3572, 14.4510, 14.5371,
        14.6208, 14.6831, 14.7062, 14.7006, 14.6963, 14.7335, 14.7507, 14.7575,
        14.7608, 14.7694, 14.7797, 14.7784, 14.7854, 14.8055, 14.8055, 

In [93]:
buffer._buffer.storage[buffer._buffer.storage['episode'] == 0]


TensorDict(
    fields={
        action: Tensor(shape=torch.Size([91, 39]), device=cuda:0, dtype=torch.float32, is_shared=True),
        episode: Tensor(shape=torch.Size([91]), device=cuda:0, dtype=torch.int64, is_shared=True),
        obs: TensorDict(
            fields={
                fixed_camera: Tensor(shape=torch.Size([91, 384, 384, 3]), device=cuda:0, dtype=torch.uint8, is_shared=True),
                success: Tensor(shape=torch.Size([91]), device=cuda:0, dtype=torch.bool, is_shared=True),
                vec_obs: Tensor(shape=torch.Size([91, 93]), device=cuda:0, dtype=torch.float32, is_shared=True)},
            batch_size=torch.Size([91]),
            device=cuda,
            is_shared=True),
        reward: Tensor(shape=torch.Size([91]), device=cuda:0, dtype=torch.float32, is_shared=True)},
    batch_size=torch.Size([91]),
    device=cuda,
    is_shared=True)

In [61]:
buffer.save_hdf5('test.h5')

AttributeError: 'LazyTensorStorage' object has no attribute 'index_select'

In [58]:
def to_td(obs, action, reward=None):
    """Creates a TensorDict for a new episode."""
    if isinstance(obs, dict):
        obs = TensorDict(obs, batch_size=(), device='cpu')
    else:
        obs = obs.unsqueeze(0).cpu()
    if reward is None:
        reward = torch.tensor(float('nan'))
    td = TensorDict(dict(
        obs=obs.unsqueeze(0),
        action=action.unsqueeze(0),
        reward=torch.tensor(reward).unsqueeze(0),
    ), batch_size=(1,))
    return td


def evaluate(cfg): 
    success_only = cfg.success_only
    assert torch.cuda.is_available()
    assert cfg.eval_episodes > 0, 'Must evaluate at least 1 episode.'
    cfg = parse_cfg(cfg)
    set_seed(cfg.seed)
    print(colored(f'Task: {cfg.task}', 'blue', attrs=['bold']))
    print(colored(f'Model size: {cfg.get("model_size", "default")}', 'blue', attrs=['bold']))
    print(colored(f'Checkpoint: {cfg.checkpoint}', 'blue', attrs=['bold']))
    if not cfg.multitask and ('mt80' in cfg.checkpoint or 'mt30' in cfg.checkpoint):
        print(colored('Warning: single-task evaluation of multi-task models is not currently supported.', 'red', attrs=['bold']))
        print(colored('To evaluate a multi-task model, use task=mt80 or task=mt30.', 'red', attrs=['bold']))

    # Make environment
    env = make_env(cfg)
    buffer = RobomimicBuffer(cfg)

    # Load agent
    agent = TDMPC2(cfg)
    assert os.path.exists(cfg.checkpoint), f'Checkpoint {cfg.checkpoint} not found! Must be a valid filepath.'
    agent.load(cfg.checkpoint)
    
    # Evaluate
    if cfg.multitask:
        print(colored(f'Evaluating agent on {len(cfg.tasks)} tasks:', 'yellow', attrs=['bold']))
    else:
        print(colored(f'Evaluating agent on {cfg.task}:', 'yellow', attrs=['bold']))
    if cfg.save_video:
        video_dir = os.path.join(cfg.work_dir, 'videos')
        os.makedirs(video_dir, exist_ok=True)
    scores = []
    tasks = cfg.tasks if cfg.multitask else [cfg.task]
    for task_idx, task in enumerate(tasks):
        if not cfg.multitask:
            task_idx = None
        ep_rewards, ep_successes = [], []
        for i in range(cfg.eval_episodes):
            obs, done, ep_reward, t = env.reset(task_idx=task_idx), False, 0, 0
            obs_dict = {"vec_obs": obs,  "success": False, "fixed_camera": env.render()}
            _tds = [to_td(obs_dict, torch.zeros_like(env.rand_act()), ep_reward)]
            if cfg.save_video:
                frames = [env.render()]
            while not done:
                action = agent.act(obs, t0=t==0, task=task_idx)
                obs, reward, done, info = env.step(action)
                obs_dict = {"vec_obs": obs,  "success": np.array(info["solved"], dtype=bool), "fixed_camera": env.render()}
                _tds.append(to_td(obs_dict, action.flatten(), reward))
                ep_reward += reward
                t += 1
                if cfg.save_video:
                    frames.append(env.render())
            
            if success_only and info["success"] or not success_only:
                buffer.add(torch.cat(_tds))

            ep_rewards.append(ep_reward)
            ep_successes.append(info['success'])
            if cfg.save_video:
                imageio.mimsave(
                    os.path.join(video_dir, f'{task}-{i}.mp4'), frames, fps=15)
        buffer.save(cfg.hdf5_save_path)
        ep_rewards = np.mean(ep_rewards)
        ep_successes = np.mean(ep_successes)
        if cfg.multitask:
            scores.append(ep_successes*100 if task.startswith('mw-') else ep_rewards/10)
        print(colored(f'  {task:<22}' \
            f'\tR: {ep_rewards:.01f}  ' \
            f'\tS: {ep_successes:.02f}', 'yellow'))
    if cfg.multitask:
        print(colored(f'Normalized score: {np.mean(scores):.02f}', 'yellow', attrs=['bold']))
    return buffer

In [ ]:
cfg.buffer_size

1000

In [2]:
repo_id = "nicklashansen/tdmpc2"
subfolder = "myosuite"

# List of checkpoint filenames to download
fs = HfFileSystem()

checkpoint_files = map(lambda x: x.split('/')[-1], fs.glob(f"{repo_id}/myosuite/*.pt"))

myo_ckpts = []
# Download each checkpoint
for file_name in checkpoint_files:
    file_path = hf_hub_download(repo_id, filename=file_name, subfolder=subfolder)
    print(f"Downloaded {file_name} to {file_path}")
    myo_ckpts.append(file_path)

subfolder_path = os.path.split(myo_ckpts[0])[0]

# subfolder = "dmcontrol"

# checkpoint_files = ['dog-run-1.pt']

# dmcontrol_ckpts = []

# for file_name in checkpoint_files:
#     file_path = hf_hub_download(repo_id, filename=file_name, subfolder=subfolder)
#     print(f"Downloaded {file_name} to {file_path}")
#     dmcontrol_ckpts.append(file_path)


Downloaded myo-hand-key-turn-1.pt to /home/krishnans/.cache/huggingface/hub/models--nicklashansen--tdmpc2/snapshots/8fb2a82efb3bae96941da440128fe1332e4394fd/myosuite/myo-hand-key-turn-1.pt
Downloaded myo-hand-key-turn-2.pt to /home/krishnans/.cache/huggingface/hub/models--nicklashansen--tdmpc2/snapshots/8fb2a82efb3bae96941da440128fe1332e4394fd/myosuite/myo-hand-key-turn-2.pt
Downloaded myo-hand-key-turn-3.pt to /home/krishnans/.cache/huggingface/hub/models--nicklashansen--tdmpc2/snapshots/8fb2a82efb3bae96941da440128fe1332e4394fd/myosuite/myo-hand-key-turn-3.pt
Downloaded myo-hand-key-turn-hard-1.pt to /home/krishnans/.cache/huggingface/hub/models--nicklashansen--tdmpc2/snapshots/8fb2a82efb3bae96941da440128fe1332e4394fd/myosuite/myo-hand-key-turn-hard-1.pt
Downloaded myo-hand-key-turn-hard-2.pt to /home/krishnans/.cache/huggingface/hub/models--nicklashansen--tdmpc2/snapshots/8fb2a82efb3bae96941da440128fe1332e4394fd/myosuite/myo-hand-key-turn-hard-2.pt
Downloaded myo-hand-key-turn-hard-3

FileNotFoundError: [Errno 2] No such file or directory: '/home/krishnans/.cache/huggingface/hub/models--nicklashansen--tdmpc2/snapshots/8fb2a82efb3bae96941da440128fe1332e4394fd/myosuite' -> 'tdmpc2_ckpts/myosuite'

In [3]:
def get_task(x):
    x = x.split('/')[-1]
    x = '-'.join(['myo'] + x.split('-')[2:-1]).rstrip('.pt')
    return x

In [6]:
set(list(map(get_task, myo_ckpts)))

{'myo-key-turn',
 'myo-key-turn-hard',
 'myo-obj-hold',
 'myo-obj-hold-hard',
 'myo-pen-twirl',
 'myo-pen-twirl-hard',
 'myo-pose',
 'myo-pose-hard',
 'myo-reach',
 'myo-reach-hard'}

In [15]:
ckpt = myo_ckpts[0]

with hydra.initialize(config_path='.'):
    cfg = hydra.compose('config.yaml',
                        return_hydra_config=True,
                        overrides=[f'task={get_task(ckpt)}', 
                                    f'checkpoint={ckpt}'])
    HydraConfig.instance().set_config(cfg)
    cfg = parse_cfg(cfg)
    cfg.buffer_size = (gym.envs.registry.spec(MYOSUITE_TASKS[cfg.task]).max_episode_steps + 1) * cfg.eval_episodes


In [50]:
ckpt_scores = defaultdict(list)

for ckpt in myo_ckpts: 
    with hydra.initialize(config_path='.'):
        cfg = hydra.compose('config.yaml',
                            return_hydra_config=True,
                            overrides=[f'task={get_task(ckpt)}', 
                                       f'checkpoint={ckpt}'])
        HydraConfig.instance().set_config(cfg)
        cfg = parse_cfg(cfg)
    ckpt_scores[get_task(ckpt)].append(evaluate(cfg))



Task: myo-key-turn
Model size: default
Checkpoint: /home/krishnans/.cache/huggingface/hub/models--nicklashansen--tdmpc2/snapshots/8fb2a82efb3bae96941da440128fe1332e4394fd/myosuite/myo-hand-key-turn-1.pt


Evaluating agent on myo-key-turn:
  myo-key-turn          	R: 1205.6  	S: 1.00
Task: myo-key-turn
Model size: default
Checkpoint: /home/krishnans/.cache/huggingface/hub/models--nicklashansen--tdmpc2/snapshots/8fb2a82efb3bae96941da440128fe1332e4394fd/myosuite/myo-hand-key-turn-2.pt
Evaluating agent on myo-key-turn:
  myo-key-turn          	R: 1217.4  	S: 1.00
Task: myo-key-turn
Model size: default
Checkpoint: /home/krishnans/.cache/huggingface/hub/models--nicklashansen--tdmpc2/snapshots/8fb2a82efb3bae96941da440128fe1332e4394fd/myosuite/myo-hand-key-turn-3.pt
Evaluating agent on myo-key-turn:
  myo-key-turn          	R: 1236.0  	S: 1.00
Task: myo-key-turn-hard
Model size: default
Checkpoint: /home/krishnans/.cache/huggingface/hub/models--nicklashansen--tdmpc2/snapshots/8fb2a82efb3bae96941da440128fe1332e4394fd/myosuite/myo-hand-key-turn-hard-1.pt
Evaluating agent on myo-key-turn-hard:
  myo-key-turn-hard     	R: 1072.6  	S: 0.00
Task: myo-key-turn-hard
Model size: default
Checkpoint: /hom

Simulation couldn't be stepped as intended. Issuing a reset
  myo-pen-twirl-hard    	R: 3702.1  	S: 0.50
Task: myo-pose
Model size: default
Checkpoint: /home/krishnans/.cache/huggingface/hub/models--nicklashansen--tdmpc2/snapshots/8fb2a82efb3bae96941da440128fe1332e4394fd/myosuite/myo-hand-pose-1.pt
Evaluating agent on myo-pose:
  myo-pose              	R: 715.9  	S: 1.00
Task: myo-pose
Model size: default
Checkpoint: /home/krishnans/.cache/huggingface/hub/models--nicklashansen--tdmpc2/snapshots/8fb2a82efb3bae96941da440128fe1332e4394fd/myosuite/myo-hand-pose-2.pt
Evaluating agent on myo-pose:
  myo-pose              	R: 715.2  	S: 1.00
Task: myo-pose
Model size: default
Checkpoint: /home/krishnans/.cache/huggingface/hub/models--nicklashansen--tdmpc2/snapshots/8fb2a82efb3bae96941da440128fe1332e4394fd/myosuite/myo-hand-pose-3.pt
Evaluating agent on myo-pose:
  myo-pose              	R: 733.6  	S: 1.00
Task: myo-pose-hard
Model size: default
Checkpoint: /home/krishnans/.cache/huggingface/h

In [120]:
ckpt = myo_ckpts[4]
with hydra.initialize(config_path='.'):
    cfg = hydra.compose('config.yaml',
                        return_hydra_config=True,
                        overrides=[f'task={get_task(ckpt)}', 
                                    f'checkpoint={ckpt}',
                                    'save_video=true'])
    HydraConfig.instance().set_config(cfg)
    cfg = parse_cfg(cfg)
    env = make_env(cfg)
    env._max_episode_steps

evaluate(cfg)

Task: myo-key-turn-hard
Model size: default
Checkpoint: /home/krishnans/.cache/huggingface/hub/models--nicklashansen--tdmpc2/snapshots/8fb2a82efb3bae96941da440128fe1332e4394fd/myosuite/myo-hand-key-turn-hard-2.pt
Evaluating agent on myo-key-turn-hard:
  myo-key-turn-hard     	R: 1168.7  	S: 0.10


(1168.6514, 0.1)

In [128]:
list(cfg.work_dir.iterdir())

[PosixPath('/home/krishnans/ngc/tdmpc2/tdmpc2/logs/myo-key-turn-hard/1/default/videos')]

In [119]:
# Display the video in the Jupyter notebook
Video(str(list((cfg.work_dir / 'videos').iterdir())[1]))

In [130]:
for i, flag in enumerate([mujoco.mjtVisFlag(i) for i in range(25)]):
    if env.sim.renderer._scene_option.flags[i]:
        print(flag)

AttributeError: 'MJRenderer' object has no attribute '_scene_option'